In [1]:
#Cntrl+Enter выполнить ячейку

#Путь к файлу экспорта из Pulseview
fname = "/home/arabella/12MHZ"

f = open(fname, 'r')
lines = f.readlines()
lines = lines[2:]
print("1st 8 lines:")
for i in range(8):
    print(lines[i])

1st 8 lines:
D1:00000000 00000000 00000000 00111111 11111111 11111111 11111111 11111111

D1:11111111 11100000 00000000 00000000 00000000 00000000 00000000 00111111

D1:11111111 11111111 11111111 11111111 11111111 11100000 00000000 00000000

D1:00000000 00000000 00000000 00111111 11111111 11111111 11111111 11111111

D1:11111111 11100000 00000000 00000000 00000000 00000000 00000000 00111111

D1:11111111 11111111 11111111 11111111 11111111 11100000 00000000 00000000

D1:00000000 00000000 00000000 00011111 11111111 11111111 11111111 11111111

D1:11111111 11100000 00000000 00000000 00000000 00000000 00000000 00011111



In [2]:
#Тут надо ввести номер канала, который ты хочешь расшифровывать. В файле 12MHZ это 1, например 
channel = input()


1


In [5]:
len(lines)

1564160

In [65]:
#Вычленяем нужный канал, удаляем пробелы, переносы строки и т п
#работает долго, секунд пять, т к файл большой

dump = ""

for line in lines[0:200000]:
    if line[0:3] == "D{}:".format(channel) or line[0:4] == "D{}:".format(channel):
        dump = dump + line.replace("\n"," ")
dump = dump.replace(" ","")
dump = dump.replace("D1:","")
print(dump[0:100])

0000000000000000000000000011111111111111111111111111111111111111111111111110000000000000000000000000


In [13]:
#находим начало синхронизирующей частоты. По сути - выбрасываем префикс из одинаковых символов

sync_pos = 0
a = dump[0]
while(dump[sync_pos] == a):
    sync_pos = sync_pos + 1
dump = dump[sync_pos:]
print(dump[0:100])

1111111111111111111111111111111111111111111111111000000000000000000000000000000000000000000000001111


In [14]:
#частота захвата в мегагерцах
freq = 12


#Хавает на вход длинк последовательности одинаковых символов, отдает ее декод из nrzi 
def parse(s):
    if(s < 3):
        return []
    if(s%(freq//2) < (freq//4)):
        x = 2*s//freq
    else:
        x = (2*s//freq) + 1
    ans = [False]
    ans.extend([True for i in range(0,x-1)])
    return ans

parse(2), parse(3), parse(8), parse(9)

([], [False], [False], [False, True])

In [15]:
# разбираем NRZ-I, преобразовываем X МГЦ в 2 МГЦ, выкидываем несущую частоту
# работает долго, секнд 20 для 12MHZ.txt


# идем по дампу.несущая частота - это ~48 нулей, затем ~48 единиц, и т д.(для 12 мгц, для произвольной - 4*freq)
# В посылках нет 8 повторяющихся так что последовательности длины 45    (4*freq - freq/4 в общем случае) 
# то что длины 45 и больше - точно несущая частота
sync_pos = 0
a = dump[0]
last_pos = 0
bitdump = []
while sync_pos < len(dump):
    a = dump[last_pos]
    sync_pos = last_pos
    while(sync_pos < len(dump) and dump[sync_pos] == a):
        sync_pos = sync_pos + 1
    if(sync_pos - last_pos <= 4*freq - freq//4):
        bitdump.extend(parse(sync_pos-last_pos))
    last_pos = sync_pos
    
print(bitdump[0:16])

#bd - то же что и bitdump, только не булевый true/false, а 0/1 
bd = ["1" if bitdump[i] else "0" for i in range(0,len(bitdump))]
bd_str = "".join(bd)
print(bd_str[0:100])

[False, True, True, True, True, True, True, False, True, True, True, True, True, False, True, True]
0111111011111011100000000111000010000111101111110011111100111111011111011100000000111000010000111101


In [8]:
# Это часть недописанная, и я не до конца уверен в том, что она работает правильно.
# Разделяет вход на посылки, умеет часть и них классифицировать и парсить.
# Предполагается, что мы начли захватывать до начала посылок


pos = 0
i = 0
debug_lvl = 0 #1 - выводить сырое сообщение, 0 - нет
cycle_count = 0;
phaze = 0;
drv_done=0

# Это устанавливается в настройках мастера, в твинкэте типа
drive_num = 1 # ожидаемое число приводов. 

#число активных выходных IDN в каждом приводе. Список есть в S-0-15/S-0-16, насколько помню.
ind_num = {1:1} #{номер привода: число идн}, например {2:2, 3:1, 1:2, 4:2}



while True:
    #Ищем два последовательных 7Е, считаем что между ними - посылка x, y - позиции начала этих 7Е
    x = bd_str.find("01111110",pos, len(bd_str))
    if x == -1:
        break
    y = bd_str.find("01111110",x + 1, len(bd_str))
    if y == -1:
        break
    
    #Отбрасываем слишком короткие бессмысленные посылки, в которых меньше 3 байт данных 
    #(в любой посылке должен быть один байт адреса и два байта FC)
    pos = y + 8
    sss = bd_str[x+8:y]
    if(len(sss) < 24):
        pos = y
        if(debug_lvl == 1):
            print("\n\n\n Too short message. RAW DATA: '\033[91m'",bd_str[x:y+8])
    else:
        #Убираем нолики, которые протокол вставляет после каждых 5 единиц('bit-stuffing')
        sss = bd_str[x:x+8] + sss.replace("111110", "11111") + bd_str[y:y+8]
        sss = [sss[8*v:8*v+8] for v in range(0, len(sss) // 8)]
        #sss - массив из строк по 8 символов ака байтов
        try:
            #Счетчик числа сказавших свое приводов
            
            if(debug_lvl == 1):
                print("\n\n\nRAW: ", " ".join(sss))
            else:
                print('\n')
                
            # MST всегда 6 байтов, адрес - 6 единиц
            if(sss[1] == '11111111' and len(sss) == 6):
                if(phaze > 1 and drv_done < drive_num):
                    print("\033[91m WARNING: not all drive replayed prevous cycle \033[0m")
                    if(phaze < 4):
                        print("\033[91m Parsing may be incorrect \033[0m")
                if(phaze > 1 and drv_done < drive_num + 1):
                    print("\033[91m WARNING: no MDT or one DT's missing prevous cycle \033[0m")
                    if(phaze < 4):
                        print("\033[91m Parsing may be incorrect \033[0m")
                print("\n\n\n========  Cycle# %d ======== start %d stop %d" % (cycle_count, x , y+8))
                print("MST: communication phaze", int(sss[2][::-1],2) )
                cycle_count = cycle_count + 1                 
                drv_done = 0
                phaze = int(sss[2][::-1],2)
                continue
                
            # DT не может быть отправлена всем сразу всем, ее адрес - не 11111111
            # В документации написано: В фазах 1 и 2 мастер "шлет индивидуальные MDT для каждого привода"
            # Я что это означает в плане поля "адрес" я так и не понял.
            
            if(sss[1] != '11111111'):
                if(phaze < 3):
                    print('Some telegram')
                    if(phaze > 1):
                        if(drv_done < drive_num):
                            print("Looks like It's DT")
                            drv_done += 1
                        else:
                            print("Looks like It's individual MDT")
                            drv_done += 1
                        
                    for i in range(len(sss)):
                        print(sss[i], end = ' ')
                    continue
                # если фаза хотя-бы третья, то можно однозначто считать что то что имеет адрес не 1 - это DT
                # и она как-то парсится
                print("DT: \n    Address: ", int(sss[1][::-1],2))
                print("    Drive Status Word: ", sss[2],sss[3])
                print("    Service channel:   ", sss[4],sss[5])
                print("    IDNS: ")
                drv_done += 1
                # Первые 5 последние 3 байта - служебные. 4 Байта на каждую IDN 
                # (некоторым вроде надо только 2, надо смотреть в документации к приводам на конкретную IDN)
                # Но я 2-х байтовых не встречал
                
                idn_start = 6
                while idn < len(sss) - 3:
                    value = sss[idn_start] + sss[idn_start+1] + sss[idn_start+2] + sss[idn_start+3]
                    idn_start = idn_start + 4
                    #переворачиваем значение (т.к. little-endian) и выводим
                    print("        ",int(value[::-1],2))
            
            # Парсер MDT не написан от слова совсем, просто выводит то что классифицируется как 
            # MDT по остаточному принципу
            else:
                drv_done += 1
                offset = 2
                print("MDT:")
                for i in range(1,drive_num + 1):
                      print('    Drive #%d'% i)
                      print('    Master Control Word: ', sss[offset], sss[offset + 1])
                      offset += 2
                      print('    Service Channel:     ', sss[offset], sss[offset + 1])
                      offset += 2
                      print("    IDNS:")
                      for idnN in range(ind_num[i]):
                          value = sss[offset] + sss[offset+1] + sss[offset+2] + sss[offset+3]
                          offset += 4
                          #переворачиваем значение (т.к. little-endian) и выводим
                          print("        ",int(value[::-1],2))

            
                
        except Excepion as e:
            print("eror during parsing:", e)
            continue






========  Cycle# 0 ======== start 0 stop 49
MST: communication phaze 0





========  Cycle# 1 ======== start 57 stop 106
MST: communication phaze 0





========  Cycle# 2 ======== start 113 stop 162
MST: communication phaze 0





========  Cycle# 3 ======== start 174 stop 223
MST: communication phaze 0





========  Cycle# 4 ======== start 227 stop 276
MST: communication phaze 0





========  Cycle# 5 ======== start 282 stop 331
MST: communication phaze 0





========  Cycle# 6 ======== start 340 stop 389
MST: communication phaze 0





========  Cycle# 7 ======== start 395 stop 444
MST: communication phaze 0





========  Cycle# 8 ======== start 453 stop 502
MST: communication phaze 0





========  Cycle# 9 ======== start 505 stop 554
MST: communication phaze 0





========  Cycle# 10 ======== start 561 stop 610
MST: communication phaze 0





========  Cycle# 11 ======== start 620 stop 669
MST: communication phaze 0





========  Cycle# 12 ======== start 673 stop 722
M

Looks like It's DT
01111110 01000000 10000000 00010101 00000000 00000000 10001010 00110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 10000000 00000000 00000001 00000000 00010001 10010010 01111110 




========  Cycle# 313 ======== start 18391 stop 18440
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010101 00000000 00000000 10001010 00110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 10000000 00000000 00000001 00000000 00010001 10010010 01111110 




========  Cycle# 314 ======== start 18602 stop 18651
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010101 00000000 00000000 10001010 00110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 10000000 00000000 00000001 00000000 00010001 10010010 01111110 




========  Cycle# 315 ======== start 18822 stop 18871
MST: communication phaze 2


Some telegram
Looks like It

01111110 01000000 11110000 00000000 11000000 00000000 01110110 10000010 01111110 




========  Cycle# 345 ======== start 25266 stop 25315
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010101 00000000 00000000 10001010 00110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11110000 00000000 11000000 00000000 01110110 10000010 01111110 




========  Cycle# 346 ======== start 25484 stop 25533
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010101 00000000 00000000 10001010 00110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 00011000 00000000 11000000 00000000 01110111 10101011 01111110 




========  Cycle# 347 ======== start 25699 stop 25748
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 00000000 00010101 10000000 00000000 01001100 10010000 01111110 

Some telegram
Looks like It's individual MDT
01111110 0100

01111110 01000000 00111100 00000000 10100000 00000000 10000001 00111110 01111110 




========  Cycle# 391 ======== start 35060 stop 35109
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 00000000 00010101 00111011 00000000 10001110 01100111 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11100000 00000000 10100000 00000000 01100110 00001111 01111110 




========  Cycle# 392 ======== start 35274 stop 35323
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010101 00111011 00000000 01010011 01011111 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11100000 00000000 10100000 00000000 01100110 00001111 01111110 




========  Cycle# 393 ======== start 35493 stop 35542
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010101 00111011 00000000 01010011 01011111 01111110 

Some telegram
Looks like It's individual MDT
01111110 0100

01111110 01000000 01110000 00000000 01010000 00000000 10110011 01010001 01111110 




========  Cycle# 456 ======== start 48970 stop 49019
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 00000000 00010101 00000000 00000000 01010111 00001000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11111100 00000000 00010000 00000000 00101100 10010111 01111110 




========  Cycle# 457 ======== start 49187 stop 49236
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010101 00000000 00000000 10001010 00110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11111100 00000000 00010000 00000000 00101100 10010111 01111110 




========  Cycle# 458 ======== start 49405 stop 49454
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010101 00000000 00000000 10001010 00110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 0100

01111110 01000000 11111100 00000000 11000000 00000000 00111001 10110000 01111110 




========  Cycle# 514 ======== start 61424 stop 61473
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000100 00010001 10010000 00000000 10000100 11101010 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11111100 00000000 11000000 00000000 00111001 10110000 01111110 




========  Cycle# 515 ======== start 61634 stop 61683
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000100 00010001 10010000 00000000 10000100 11101010 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 01100000 00000000 11000000 00000000 10110000 00011101 01111110 




========  Cycle# 516 ======== start 61850 stop 61899
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 00000100 00010001 11000000 00000000 01010111 01101101 01111110 

Some telegram
Looks like It's individual MDT
01111110 0100

01111110 01000000 10000000 00010001 00000000 00000000 01010110 11110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11100000 00000000 11111110 00000000 01001011 10111111 01111110 




========  Cycle# 586 ======== start 76918 stop 76967
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010001 00000000 00000000 01010110 11110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11100000 00000000 11111110 00000000 01001011 10111111 01111110 




========  Cycle# 587 ======== start 77132 stop 77181
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010001 00000000 00000000 01010110 11110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11100000 00000000 11111110 00000000 01001011 10111111 01111110 




========  Cycle# 588 ======== start 77347 stop 77396
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 0100

01111110 01000000 10000000 00010001 00000000 00000000 01010110 11110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11100000 00000000 11111110 00000000 01001011 10111111 01111110 




========  Cycle# 648 ======== start 90319 stop 90368
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010001 00000000 00000000 01010110 11110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11100000 00000000 11111110 00000000 01001011 10111111 01111110 




========  Cycle# 649 ======== start 90543 stop 90592
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 01000000 10000000 00010001 00000000 00000000 01010110 11110000 01111110 

Some telegram
Looks like It's individual MDT
01111110 01000000 11100000 00000000 11111110 00000000 01001011 10111111 01111110 




========  Cycle# 650 ======== start 90761 stop 90810
MST: communication phaze 2


Some telegram
Looks like It's DT
01111110 0100

NameError: name 'Excepion' is not defined

In [84]:
print("\033[3m\033[33m\033[41m{}\033[0m".format("Htua_0111100000"))
print('abc')

Htua_0111100000
abc
